## Activity - 1

* Organizing the Images into Different Classes.

In [2]:
import os
import shutil
import sys
import pandas as pd

In [3]:
dataset_dir = '/content/drive/MyDrive/al_course/content/train'
labels = pd.read_csv('/content/drive/MyDrive/al_course/content/labels.csv')

In [4]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
import os
def make_dir(x):
 if os.path.exists(x) == False:
     os.makedirs(x)
base_dir = '/content/drive/MyDrive/al_course/content/subset'
make_dir(base_dir)

In [6]:
train_dir = os.path.join(base_dir, 'train')
make_dir(base_dir)

breeds = labels.breed.unique()

In [ ]:
breeds = labels.breed.unique()
for breed in breeds:
    #Make folder for each breed
    _ = os.path.join(train_dir, breed)
    make_dir(_)

    #Copy images to the crresponsing folder
    images = labels[labels.breed == breed]['id']
    for image in images:
        source = os.path.join(dataset_dir, f'{image}.jpg')
        destination = os.path.join(train_dir, breed, f'{image}.jpg')
        try:
          shutil.copyfile(source,destination)
        except:
          pass

## Activity - 2

* Import the ImageDataGenerator library.

In [77]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Activity - 3

* Configure ImageDataGenerator class

In [78]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)

## Activity - 4

* Apply ImageDataGenerator functionality to Trainset and Test set

In [79]:
datagen = ImageDataGenerator()
generator = datagen.flow_from_directory(
    "/content/drive/MyDrive/al_course/content/subset/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False,
    classes=breeds.tolist(),
)

Found 8828 images belonging to 120 classes.


# Model Building

## Activity - 1
* Importing the Model Building Libraries

In [80]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import softmax
from keras.api._v2.keras import activations

## Activity - 2
* Importing the  VGG19 model

In [81]:
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

## Activity - 3
* Initializing the model

In [82]:
Image_size=[224, 224]

In [83]:
sol=VGG19(input_shape=Image_size + [3], weights='imagenet', include_top=False)

In [84]:
for i in sol.layers:
  i.trainable = False

In [85]:
y=Flatten()(sol.output)

## Activity - 4
* Adding Fully connected Layers

In [94]:
from keras.api._v2.keras import activations
final = Dense(20, activation='softmax')(y)

In [95]:
sol.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## Activity - 5
* Configure The Learning Process

In [99]:
sol.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['Accuracy'])

## Activity - 6
* Train the Model

In [100]:
sol.fit(generator, epochs=6)

Epoch 1/6


ValueError: ignored